In [5]:
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'c:\\Users\\602870\\Desktop\\gamechanger\\gamechanger')) # '.' if the path is to current folder
except:
    pass

import pandas as pd
import numpy as np
import re
import spacy as sp
import time

# from dataScience.src.utilities.text_utils import simple_clean
import dataScience.src.utilities.spacy_model as spacy_
from dataScience.src.utilities.text_utils import simple_clean
from dataScience.src.featurization.extract_improvement.extract_utils import extract_entities, create_list_from_dict, remove_articles, match_parenthesis

spacy_model = spacy_.get_lg_nlp()

In [ ]:
# test = ['USD(C CFO', 'DoDD', 'DeCA', 'Department of Defense', 'CFO', 'Defense (Comptroller Chief Financial Officer', 'Defense for Policy (']
# match_parenthesis(test)

In [12]:
test = 'Manage and conduct DIA\u2019s worldwide Defense HUMINT collection operations in \nsupport of DoD and IC requirements, provide direct access to critical military information, and \nenable other intelligence collection disciplines in accordance with DoD Directives S-5105.29 and \n5200.37 (References (r) and (s)).'
test = re.sub("[\\n\\t\\r]+", "", test)
print(test)

Manage and conduct DIA’s worldwide Defense HUMINT collection operations in support of DoD and IC requirements, provide direct access to critical military information, and enable other intelligence collection disciplines in accordance with DoD Directives S-5105.29 and 5200.37 (References (r) and (s)).


In [ ]:
df["ref"] = ""
for i, row in df.iterrows():
    for j in list(df.columns):
        if isinstance(row[j],str):
            if ":" in row[j]:
                if "shall" in row[j]:
                    entity = row[j].split(":",1)[0]
                    entity = entity.split("shall",1)[0]
                    entity = entity.split(".")[-1].strip()

                    df.at[i,'entity'] = entity

    agencies = []
    refs = []
    for j in list(df.columns):

        if type(row[j]) == str:
            if j!= "doc":
                refs.append(list(collect_ref_list(row[j]).keys()))
                agencies.append([aliases[x] for x in aliases.keys() if " "+str(x) in row[j]])
    flat_a = [item for sublist in agencies for item in sublist]
    flat_a = set(flat_a)
    df.at[i,'agencies'] = list(flat_a)
    flat_r = [item for sublist in refs for item in sublist]
    flat_r = list(set(flat_r))
    df.at[i,'ref'] = flat_r

In [ ]:
data = pd.read_csv('dataScience/src/featurization/extract_improvement/data/pipe.csv')
data['agencies'].replace(np.nan, '', regex=True, inplace=True)
combined_cols = pd.DataFrame(data[data.columns[1:]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1), columns=['text'])
combined_cols_list = combined_cols['text'].tolist()

In [2]:
start = time.time()
per_doc_list = []
all_docs = []

for i in range(len(combined_cols_list)):
    sentence = combined_cols_list[i]
    entities = extract_entities(sentence, spacy_model)
    
    # if i % 200 == 0:
    #     print('Record number: ', i, round(time.time() - start, 2), 'seconds')

    #TODO replace when integrating into responsibilties pipeline
    prev_agencies = [x.strip() for x in data['agencies'][i].split(',')]
    prev_agencies = [i for i in prev_agencies if i]
    
    flat_entities = create_list_from_dict(entities)
    for j in prev_agencies:
        flat_entities.append(j)
    
    flat_entities = remove_articles(flat_entities)
    flat_entities = match_parenthesis(flat_entities)
    flat_entities = '|'.join(i for i in set(flat_entities))
    all_docs.append(flat_entities)

data['agencies'] = all_docs
data.drop('Unnamed: 0', axis=1, inplace=True)
print()
print('Elapsed seconds:', time.time() - start)

Record number:  0 0.05 seconds
Record number:  200 16.64 seconds
Record number:  400 25.5 seconds
Record number:  600 32.08 seconds
Record number:  800 42.68 seconds
Record number:  1000 54.38 seconds
Record number:  1200 67.41 seconds
Record number:  1400 74.24 seconds
Record number:  1600 79.06 seconds
Record number:  1800 83.55 seconds
Record number:  2000 87.78 seconds
Record number:  2200 93.81 seconds
Record number:  2400 98.05 seconds
Record number:  2600 103.8 seconds
Record number:  2800 108.27 seconds
Record number:  3000 112.7 seconds

Elapsed seconds: 114.97195935249329


In [ ]:
# data.loc[data['doc'] == 'DODD 1000.21E CH 2.json']
data.to_csv('dataScience/src/featurization/extract_improvement/model_outputs/full_pipe_w_new_entities_030221.csv')